In [ ]:
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', None)

In [ ]:
folders = ['Step1','Step2','Step3','Step4','Step5','Step6_Barcode','Step6_Indel','Step7','Step8','Step8.5','Excel','Step9','Normalized_Matrix']
for folder in folders:
    if not os.path.exists(folder):
        os.mkdir(folder)

In [ ]:
# Move all raw fastq files to Step1 folder
# If unziped, comment out the following line
# !gunzip ./Step1/*.gz
files = [f'./Step1/{f}' for f in listdir('./Step1/') if isfile(join('./Step1/', f)) and f.endswith('.fastq')]
for file in files:
    os.system(f'awk \'NR%4 ==2\' {file} > ./Step2/{file[file.rfind("/")+1:]}')

In [ ]:
files = [f'./Step2/{f}' for f in listdir('./Step2/') if isfile(join('./Step2/', f))]
for file in files:
    os.system(f'sed -e "s/^.//g" {file} > ./Step3/{file[file.rfind("/")+1:]}')

In [ ]:
files_f = [f'./Step3/{f}' for f in listdir('./Step3/') if isfile(join('./Step3/', f)) and f.find("R1") != -1]
for file in files_f:
    file1 = file
    file2 = file.replace("R1", "R2")
    os.system(f"paste -d '\\0' {file1} {file2} | cat> ./Step4/{file[file.rfind('/')+1:]}")

In [ ]:
files = [f'./Step4/{f}' for f in listdir('./Step4/') if isfile(join('./Step4/', f))]
for file in files:
    print(file[file.rfind('/')+1:])
    os.system(f'grep "^CTTATATTCCCAGGGCCGGTTCGCGATCGCCCTGCAGG[A-Z][A-Z][A-Z][A-Z][A-Z]TAGTTATTAATGACTCACGGGGATTTCCAAGTCTCCACCCCATTGACGTCAATGGG.*CCGCCCTCGACCGCCTTGATTCTCATGGTCTGGGTGC[A-Z]*GTGGTGGTTGTTCACGGTGCCCT" {file} > ./Step5/{file[file.rfind("/")+1:]}')

In [ ]:
#raw barcode and indel
files = [f'./Step5/{f}' for f in listdir('./Step5/') if isfile(join('./Step5/', f))]
for file in files:
    print(file[file.rfind('/')+1:])
    os.system(f'sed -e "s/CTTATATTCCCAGGGCCGGTTCGCGATCGCCCTGCAGG\(.*\)TAGTTATTAATGACTCACGGGGATTTCCAAGTCTCCACCCCATTGACGTCAATGGG.*CCGCCCTCGACCGCCTTGATTCTCATGGTCTGGGTGC[A-Z]*GTGGTGGTTGTTCACGGTGCCCT[A-Z]*/\\1/"  {file} > ./Step6_Barcode/{file[file.rfind("/")+1:]}')
    os.system(f'sed -e "s/CTTATATTCCCAGGGCCGGTTCGCGATCGCCCTGCAGG[A-Z][A-Z][A-Z][A-Z][A-Z]TAGTTATTAATGACTCACGGGGATTTCCAAGTCTCCACCCCATTGACGTCAATGGG.*CCGCCCTCGACCGCCTTGATTCTCATGGTCTGGGTGC\(.*\)GTGGTGGTTGTTCACGGTGCCCT[A-Z]*/\\1/"  {file} > ./Step6_Indel/{file[file.rfind("/")+1:]}')

In [ ]:
#barcode indel combo
files_barcode = [f'./Step6_Barcode/{f}' for f in listdir('./Step6_Barcode/') if isfile(join('./Step6_Barcode/', f))]
files_barcode.sort()
files_indel = [f'./Step6_Indel/{f}' for f in listdir('./Step6_Indel/') if isfile(join('./Step6_Indel/', f))]
files_indel.sort()
for file_barcode, file_indel in zip(files_barcode, files_indel):
    print(file_barcode[file_barcode.rfind('/')+1:])
    os.system(f"paste -d '\\0' {file_barcode} {file_indel} | cat > ./Step7/{file_barcode[file_barcode.rfind('/')+1:]}")

In [ ]:
#indel filter
files = [f'./Step7/{f}' for f in listdir('./Step7/') if isfile(join('./Step7/', f))]
for file in files:
    print(file[file.rfind('/')+1:])
    os.system(f"grep -v -x '.\{{45\}}' {file}  | cat >  ./Step8/{file[file.rfind('/')+1:]}")

In [ ]:
normalizer = [f'./Step5/{f}' for f in listdir('./Step5/') if isfile(join('./Step5/', f)) and f.find("R1") != -1]
normalizer.sort()
barcode_indels = [f'./Step8/{f}' for f in listdir('./Step8/') if isfile(join('./Step8/', f)) and f.find("R1") != -1]
barcode_indels.sort()
for file,normal in zip(barcode_indels,normalizer):
    writer = pd.ExcelWriter(f'./Excel/{file[file.rfind("/")+1:]}.xlsx', engine='openpyxl') 
    sheet_names = ['Barcode','Indel','Barcode_Indel']
    df = pd.read_csv(file, header=None)
    normal = pd.read_csv(normal, header=None)
    barcode_indel_freq = df.value_counts()
    barcode_indel_str = pd.DataFrame(list(barcode_indel_freq.index))
    barcode_indel_freq = barcode_indel_freq.reset_index(drop=True)
    df = pd.DataFrame()
    df['Barcode'] = barcode_indel_str[0].str[:5] 
    df['Indel'] = barcode_indel_str[0].str[5:]
    df['Freq'] = barcode_indel_freq/len(normal.index)*1000000
    df.to_excel(writer, sheet_name=f'0',index=False)
    df.to_csv(f'./Step9/{file[file.rfind("/")+1:]}',index=False)
    writer.save()

In [ ]:
def reverse_complement(seq):
    seq_dict = {'A':'T','T':'A','G':'C','C':'G','N':'N'}
    return "".join([seq_dict[base] for base in reversed(seq)])

In [ ]:
normed_freqs = [f'./Step9/{f}' for f in listdir('./Step9/') if isfile(join('./Step9/', f))]
normed_freqs.sort()
for file in normed_freqs:
    writer = pd.ExcelWriter(f'./Normalized_Matrix/{file[file.rfind("/")+1:]}.xlsx', engine='openpyxl') 
    table = pd.read_csv(file)
    table['Indel'] = table['Indel'].apply(reverse_complement)
    table_pivot = table.pivot_table(index=['Barcode'],columns=['Indel'], values=['Freq'], aggfunc='sum')
    barcode = table[['Barcode','Freq']]
    barcode = barcode.groupby('Barcode').sum()
    barcode.sort_values(by='Freq', ascending=False, inplace=True)
    indel = table[['Indel','Freq']]
    indel = indel.groupby('Indel').sum()
    indel.sort_values(by='Freq', ascending=False, inplace=True)
    table_pivot = table_pivot.fillna(0)
    table_pivot = table_pivot['Freq']
    table_pivot = table_pivot.reindex(barcode.index, axis=0)
    table_pivot = table_pivot.reindex(indel.index,axis=1)
    #Write table_pivot to excel
    table_pivot.to_excel(writer, sheet_name=f'0')
    writer.save()